In [1]:
########## SET-UP ##########

# necessary libraries
import os
import json
import requests
import time
import datetime

# INSERT YOUR API KEY HERE! DO NOT USE MINE! (Sean)
KEY = 'b00c992b-6fc1-4795-abcc-7db85c1b72fa'


In [2]:
# MATCH-v2.2
REGION_ENDPOINT = "https://{0}.api.pvp.net/api/lol/{0}/"

# FUNCTION TO PULL MATCH DATA
def get_match(REGION, matchId, includeTimeline):
    """
    Retrieve match by match ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.2/match/{1}?"
         "api_key={2}&includeTimeline={3}").
        format(REGION, matchId, KEY, includeTimeline))

In [42]:
# Constants
STATUS_OK = 200
STATUS_RATE_LIMIT_EXCEEDED = 429
valid_rank_games = ['RANKED_SOLO_5x5',  # Ranked Solo 5v5 games
                    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
                    'RANKED_TEAM_5x5']  # Ranked Team 5v5 games

REGION_list = ['na', 'kr', 'eune', 'euw', 'lan', 'las', 'oce', 'ru', 'br', 'tr']

# Sean
regional_match_id = {'br': 667074079,
 'eune': 1297479795,
 'euw': 2416870584,
 'kr': 2192858932,
 'lan': 229961444,
 'las': 273073649,
 'na': 2030002863,
 'oce': 113873464,
 'ru': 84053941,
 'tr': 325247614}

# COMMENT OUT regional_match_id THAT IS NOT UNDER YOUR NAME!!!!!!!!!!!!!!!!!!
# DO NOT RUN THIS PART OF THE CODE AFTER YOU'VE STARTED THE LOOP BELOW. IT'LL OVERWRITE EXISTING DATA!!!

# jaemin
#regional_match_id = {'na': 2029960687,
#                     'kr': 2192916925,
#                     'br': 664106032,
#                     'eune': 1293871277,
#                     'euw': 2417718483,
#                     'lan': 229932736,
#                     'las': 271679926,
#                     'oce': 113897495,
#                     'ru': 83927963,
#                     'tr': 332060998}
                     
# kenny
#regional_match_id = {'na': 2029894256
#                     'kr': 2181887785
#                     'br': 664332857
#                     'eune': 1293801833
#                     'euw': 2417655730
#                     'lan': 229928986
#                     'las': 272582143
#                     'oce': 113887342
#                     'ru': 83706280
#                     'tr': 332046148}

# andrew
#regional_match_id = {'na': 2029848145
#                     'kr': 2181825342
#                     'br': 664520626
#                     'eune': 1293842842
#                     'euw': 2416827654
#                     'lan': 229859020
#                     'las': 272319645
#                     'oce': 113877802
#                     'ru': 84139484
#                     'tr': 331322838}

# andrew_new
# regional_match_id = {'na': 2026960869,
#                     'kr': 2191725864,
#                     'br': 657525985,
#                     'eune': 1296975996,
#                     'euw': 2402323438,
#                     'lan': 230046764,
#                     'las': 272799452,
#                     'oce': 114327933,
#                     'ru': 83843684,
#                     'tr': 332012773}



In [ ]:
# Functions that check status of the data pulls
def isValid(match):
    if match.status_code != STATUS_OK:
        return False
    return True

def isClassicMatch(match):
    j = match.json()
    if j['mapId'] == 11 and j['matchMode'] == "CLASSIC" and j['matchType'] == 'MATCHED_GAME':
        return True
    else:
        return False

def isRankMatch(match):
    j = match.json()
    if j['queueType'] in valid_rank_games:
        return True
    else:
        return False
                    
def rateLimitExceeded(match):
    if int(match.status_code) == STATUS_RATE_LIMIT_EXCEEDED:
        return True
    return False

In [48]:
# initializing indices
i, n, r = (0, 0, 0) # i = incrementing starting match ID, n = number of games retrieved, r = region list index)


# feel free to stop/run this loop anytime (it remembers the last starting point.. but it doesn't hurt to double check)
for k in range(100000):
    # current region
    curr_region = REGION_list[r]
    current_match_id = regional_match_id[curr_region] + i
    i += 1
    
    m = get_match(curr_region, current_match_id, False)
    
    if not rateLimitExceeded(m):
        if isValid(m) and isClassicMatch(m) and isRankMatch(m):

            f = open("dump_global/" + str(curr_region) + str(current_match_id) + ".json", 'w+')
            # if your name is Jaemin, use this part
            #f = open("dump_jaemin_global/" + str(current_match_id)+".json", 'w+')
            # if your name is Kenny, use this part
            #f = open("dump_kenny_global/" + str(current_match_id)+".json", 'w+')
            # if your name is Andrew, use this part
            #f = open("dump_andrew_global/" + str(current_match_id)+".json", 'w+')
            f.write(m.text.encode('utf-8'))
            n += 1
            print 'n: %d, ts: %s, ID: %d, date: %s, season: %s, tier: %s, loop_index: %d, region: %s' % (n, datetime.datetime.fromtimestamp(time.time()).strftime('%H:%M:%S'), 
                                                                                                 current_match_id, 
                                                                                                 ('%s' % (time.strftime('%m-%d', time.localtime(m.json()['matchCreation']/1000)))),
                                                                                                 m.json()['season'], m.json()['participants'][1]['highestAchievedSeasonTier'], k,
                                                                                                            m.json()['region'])
            f.close()
    else:
        regional_match_id[curr_region] =  current_match_id + i
        if (r == 9):
            print ("rate limit reached at %s, moving onto %s") % (REGION_list[r], REGION_list[0])
            r = 0
        else:
            print ("rate limit reached at %s, moving onto %s") % (REGION_list[r], REGION_list[r+1])
            r += 1
        i = 0
        # My computer was slow at writing JSON files, so it took 10 seconds to come back to the same region
        # If you're computer is super fast, uncomment the time.sleep below (you probably shouldn't)
        #time.sleep(1)

n: 1, ts: 20:53:41, ID: 2029999735, date: 12-06, season: PRESEASON2016, tier: GOLD, loop_index: 2, region: NA
n: 2, ts: 20:53:41, ID: 2029999736, date: 12-06, season: PRESEASON2016, tier: SILVER, loop_index: 3, region: NA
n: 3, ts: 20:53:42, ID: 2029999737, date: 12-06, season: PRESEASON2016, tier: BRONZE, loop_index: 4, region: NA
n: 4, ts: 20:53:42, ID: 2029999738, date: 12-06, season: PRESEASON2016, tier: SILVER, loop_index: 5, region: NA
n: 5, ts: 20:53:43, ID: 2029999739, date: 12-06, season: PRESEASON2016, tier: UNRANKED, loop_index: 6, region: NA
rate limit reached at na, moving onto kr
n: 6, ts: 20:53:45, ID: 2192854202, date: 12-06, season: PRESEASON2016, tier: GOLD, loop_index: 11, region: KR
n: 7, ts: 20:53:46, ID: 2192854203, date: 12-06, season: PRESEASON2016, tier: GOLD, loop_index: 12, region: KR
n: 8, ts: 20:53:48, ID: 2192854205, date: 12-06, season: PRESEASON2016, tier: SILVER, loop_index: 14, region: KR
n: 9, ts: 20:53:51, ID: 2192854208, date: 12-06, season: PRESEAS

KeyboardInterrupt: 

{'br': 667074079,
 'eune': 1297479795,
 'euw': 2416870584,
 'kr': 2192858932,
 'lan': 229961444,
 'las': 273073649,
 'na': 2030002863,
 'oce': 113873464,
 'ru': 84053941,
 'tr': 325247614}